### Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import urllib.parse 

### Put all together

In [2]:
# create dataframe (in the beginning it is empty)
df_restaurant = pd.DataFrame(columns=['Restaurant Name', 'Address', 'Phone', 'Email', 'Homepage', 'Info'])

# sequence of numbers - 1 to 3 (4 is not included)
for i in range(1,4):

    # varibale for pages withing website
    website = 'https://www.yellowpages.com/search?search_terms=restaurant&geo_location_terms=New%20York%2C%20NY&page=' + str(i)

    # request
    response = requests.get(website)
    
    # create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # result container
    result_container = soup.find_all('div', {'class':'result'})
    
    # URL Part 1
    url_part_1 = 'https://www.yellowpages.com/'
    
    url_part_2 = []

    # loop through results
    for item in result_container:
        
    # loop through links
        for link in item.find_all('a', {'class': 'business-name'}):
            url_part_2.append(link.get('href'))
    
    # Join URL 1 and URL 2 in order to get the abosolute URL
    url_joined = []

    for link_2 in url_part_2:
        url_joined.append(urllib.parse.urljoin(url_part_1, link_2))
        
        
    # loop through all joined links
    for link in url_joined:
        response = requests.get(link)

        # create soup object
        soup = BeautifulSoup(response.content, 'html.parser')

        # name
        try:
            name = soup.find('h1').get_text()
        except:
            name = 'n/a'

        # address 
        try:
            address = soup.find('h2', {'class': 'address'}).get_text()
        except:
            address = 'n/a'

        # phone
        try:
            phone = soup.find('p', {'class':'phone'}).get_text()
        except:
            phone = 'n/a'

        # email
        try:
            email = soup.find('a', {'class': 'email-business'}).get('href').split('mailto:')[1]
        except:
            email = 'n/a'

        # website
        try:
            website = soup.find('a', {'class': 'website-link'}).get('href')
        except:
            website = 'n/a'

        # general info
        try:
            info = soup.find('dd', {'class': 'general-info'}).get_text()
        except:
            info = 'n/a'
            
        # Pandas Dataframe
        df_restaurant = df_restaurant.append({'Restaurant Name': name, 'Address':address, 'Phone': phone,
                                             'Email':email, 'Homepage': website, 'Info': info}, ignore_index=True)
    


### Output Pandas Dataframe

In [3]:
df_restaurant

,Restaurant Name,Address,Phone,Email,Homepage,Info
0,Mr. K's,"570 Lexington AveNew York, NY 10022",(212) 583-1668,info@movingmaninc.com,http://www.mrksny.com,We offer exclusive seating to you and your par...
1,Potjanee Thai Restaurant of New York,"48 Carmine StNew York, NY 10014",(212) 558-9071,mthitna2011@gmail.com,http://potjanee.net,If you're looking for authentic Thai food in N...
2,Spoonfed New York Country,"331 W 51st StNew York, NY 10019",(718) 789-3048,randy@spoonfednyc.com,http://www.spoonfed.nyc/contact-us.html,n/a
3,Da Noi Midtown Manhattan,"214 E 49th StNew York, NY 10017",(347) 955-0067,info@danoinyc.com,http://danoinyc.com/midtown-manhattan.html,"High quality dining, elegant private parties a..."
4,Seven's Turkish Grill,"158 W 72nd StNew York, NY 10023",(212) 724-4700,n/a,n/a,Here at Seven's Mediterranean Turkish Grill we...
...,...,...,...,...,...,...
85,Indian Taj,"181 Bleecker StNew York, NY 10012",(212) 982-0810,info@indiantajny.com,http://indiantajny.com,n/a
86,Rocco's Pizza Joint,"162 7th AveNew York, NY 10011",(212) 633-9555,Cola148@aol.com,http://www.roccospizzajointnyc.com,"NYC's Best Pizza. Specialty Coffee, Chelsea, NYC"
87,Juniors,"1515 BroadwayNew York, NY 10036",(212) 302-2000,n/a,https://www.juniorscheesecake.com/blog/restaur...,n/a
88,Moonstruck East Restaurant,"449 3rd AveNew York, NY 10016",(212) 213-1100,n/a,http://moonstruck.nyc/locations/449-3rd-ave,n/a


### Store in Excel

In [4]:
df_restaurant.to_excel('results_multiple_pages.xlsx', index=False)